In [1]:
import os
import json

from pprint import pprint

from langchain import LLMChain, PromptTemplate
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor, AgentType, initialize_agent
from langchain.llms import vertexai
from langchain.tools.base import BaseTool
from langchain.callbacks.manager import (AsyncCallbackManagerForToolRun, CallbackManagerForToolRun)


from vertexai.preview.language_models import TextGenerationModel
from google.cloud import aiplatform

from pydantic import BaseModel, Field, ConfigDict
from typing import Optional, Type

In [2]:
# trace langchain execution
os.environ["LANGCHAIN_TRACING"] = "false"

# authenticate 
GOOGLE_PROJECT_ID = os.getenv("GOOGLE_PROJECT_ID") # ie, confident-jackle-123456
aiplatform.init(project=GOOGLE_PROJECT_ID, location="us-central1")

## overview
In order to maintain customer privacy and minimize the ability for this agent to leak information, it is not going to have any memory (in the prompt). It is still going to be able to retrieve and relay a user's order, but that will be information retrieved from an api using a tool. 

This saves alot of tokens being fed into the input prompt

In [3]:
from tools import OrderTool, GetDetailedMenuTool, FindItemIdTool, MakeOrderCheckoutTool

tools = [GetDetailedMenuTool(), FindItemIdTool(), OrderTool(), MakeOrderCheckoutTool()]

In [4]:
agent = initialize_agent(
    tools,
    vertexai.VertexAI(temperature=0),
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [7]:
agent.run("could I check out?")



> Entering new AgentExecutor chain...
 Action:
```
{
  "action": "make_order_checkout",
  "action_input": {}
}
```

Observation: ERROR: square create_order call threw an error.
Thought: Action:
```
{
  "action": "Final Answer",
  "action_input": "Sorry, there was an error processing your request. Please try again."
}
```

> Finished chain.


'Sorry, there was an error processing your request. Please try again.'

## Chirp
TODO: setup google chirp just for a simple demo